# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,25.03,100,75,3.60,CK,1690136541
1,1,puerto armuelles,8.2833,-82.8667,31.99,77,92,1.34,PA,1690136541
2,2,bethel,41.3712,-73.4140,30.41,47,0,4.12,US,1690136542
3,3,san fernando,15.0286,120.6898,25.85,91,100,0.42,PH,1690136542
4,4,grytviken,-54.2811,-36.5092,-6.70,73,15,4.67,GS,1690136542


---


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_humidity = city_data_df.hvplot.points("Lng","Lat",
                                          xlabel="Longitude",
                                          ylabel="Latitude",
                                          size="Humidity",
                                          color="City",
                                          title="Humidity Map",
                                          geo=True,
                                          hover_cols=["City"],
                                          tiles="OSM"                                          
                                         )                                        

# Display the map
map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cond_df = city_data_df[(city_data_df['Max Temp'] > 22)
                                     &(city_data_df['Max Temp'] < 30)
                                     &(city_data_df['Humidity'] < 50)
                                     &(city_data_df['Wind Speed'] < 12)
                                    ].dropna()

# Display sample data
ideal_weather_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,kapuskasing,49.4169,-82.4331,27.05,47,75,5.66,CA,1690136543
25,25,kabalo,-6.0500,26.9167,28.29,32,8,1.96,CD,1690136545
81,81,san julian,21.0167,-102.1667,24.74,39,100,4.17,MX,1690136555
109,109,baker city,44.7749,-117.8344,28.66,28,0,2.57,US,1690136560
126,126,lusambo,-4.9667,23.4500,26.84,30,8,0.87,CD,1690136563


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cond_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = "" 

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,kapuskasing,CA,49.4169,-82.4331,47,
25,kabalo,CD,-6.0500,26.9167,32,
81,san julian,MX,21.0167,-102.1667,39,
109,baker city,US,44.7749,-117.8344,28,
126,lusambo,CD,-4.9667,23.4500,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
limit = 1
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

        # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    hotel_request = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel_response = hotel_request.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

Starting hotel search
kapuskasing - nearest hotel: Travelodge
kabalo - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
baker city - nearest hotel: Geiser Grand Hotel
lusambo - nearest hotel: No hotel found
balqash - nearest hotel: Престиж
winsted - nearest hotel: No hotel found
senirkent - nearest hotel: Aydın Tola Kız Öğrenci Apartı
zhangatas - nearest hotel: Арман
nuevo ideal - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
eagle point - nearest hotel: Brookside
saint-francois - nearest hotel: Chez Lily
dinar - nearest hotel: Otel Asya
atasu - nearest hotel: Диана
quetigny - nearest hotel: Kyriad
heniches'k - nearest hotel: Орхідея
ismayilli - nearest hotel: Otel Niyal
elliot lake - nearest hotel: Hampton Inn Elliot Lake
calama - nearest hotel: Apart Hotel Geotel Calama
kashgar - nearest hotel: 深航国际酒店
lucapa - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
yungay - nearest hotel: Rima Rima
panjakent - 

,City,Country,Lat,Lng,Humidity,Hotel Name
9,kapuskasing,CA,49.4169,-82.4331,47,Travelodge
25,kabalo,CD,-6.0500,26.9167,32,No hotel found
81,san julian,MX,21.0167,-102.1667,39,No hotel found
109,baker city,US,44.7749,-117.8344,28,Geiser Grand Hotel
126,lusambo,CD,-4.9667,23.4500,30,No hotel found
130,balqash,KZ,46.8481,74.9950,32,Престиж
143,winsted,US,41.9212,-73.0601,47,No hotel found
145,senirkent,TR,38.1044,30.5486,33,Aydın Tola Kız Öğrenci Apartı
150,zhangatas,KZ,43.5667,69.7500,21,Арман
201,nuevo ideal,MX,24.8875,-105.0728,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points("Lng","Lat",
                                          xlabel="Longitude",
                                          ylabel="Latitude",
                                          size="Humidity",
                                          color="City",
                                          title="Hotel Map",
                                          geo=True,
                                          hover_cols=["City","Country","Hotel Name"],
                                          tiles="OSM"                                          
                                         )                                        

# Display the map
map_hotel


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)